In [48]:
import pandas as pd
from bs4 import BeautifulSoup

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
import requests

import pathlib

import time


In [49]:
#Create driver instance:
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))


[WDM] - Downloading: 100%|██████████| 6.46M/6.46M [00:01<00:00, 4.19MB/s]


In [50]:
# Define path for secrets.txt:

DATA_PATH =  pathlib.Path()

with open(DATA_PATH.joinpath('secrets.txt')) as f:
    secrets = f.read().splitlines() 

In [51]:
# Open the website:
driver.get('https://campus-tools.ironhack.com/#/cohorts/618138e279542a002c897068/show/course_progress/assignments')



In [52]:
# E-mail box:
email_box = driver.find_element(By.ID, "identifierId")

email_box.clear()

email_box.send_keys(secrets[0])



In [53]:
# Advance button:
advance_1 = driver.find_element(By.ID, "identifierNext")

advance_1.click()

In [54]:
# Password box:
pass_box = driver.find_element(By.NAME, "password")
time.sleep(0.5)
pass_box.clear()
time.sleep(1.5)
pass_box.send_keys(secrets[1])


In [55]:
# Advance button:
advance_2 = driver.find_element(By.ID, "passwordNext")
time.sleep(1.5)
advance_2.click()

In [56]:
# Get student portal link again because sometimes the previous page gets stuck
driver.get('https://campus-tools.ironhack.com/#/cohorts/618138e279542a002c897068/show/course_progress/assignments')


In [57]:
#Find all rows of Weeks Separators and store in variable:

weeks_table = driver.find_elements(By.CSS_SELECTOR,"tr[class^='MuiTableRow-root']")



In [76]:
#DataFrame where we'll store our students labs info:
labs_overall = pd.DataFrame( columns= ["week", "lab_name", "student_name", "lab_status", "delivery_date"])

#Iterate through all weeks, start at 1 because, index 0 corresponds to the header:
for week_number in range(len(weeks_table)):
    if week_number < 1:
        pass
    else: 
        try:

            #Drop down Weeks separator to see the labs:
            drop_down_weeks = weeks_table[week_number].find_element(By.CSS_SELECTOR,"div[class^='MuiButtonBase-root']")
            drop_down_weeks.click()
            time.sleep(3)

            #Get week number string:
            week = weeks_table[week_number].find_element(By.CSS_SELECTOR, "td[class^='MuiTableCell-root MuiTableCell-body column-display_name']").text
            print(week)


            #Gets the html content inside the week in search :
            labs_table =   driver.find_elements(By.CSS_SELECTOR,"td[class^='MuiTableCell-root MuiTableCell-body MuiTableCell-sizeSmall']")
            
            #Get lab name and directions of dropdown of labs:
            lab_names = labs_table[0].find_elements(By.CLASS_NAME, "MuiBadge-root")
            drop_down_labs = labs_table[0].find_elements(By.CSS_SELECTOR,"div[class^='MuiButtonBase-root']")

            #Go for each lab inside the respective week:
            for index in range(len(lab_names)):
                each_lab_name = lab_names[index].text
                print(each_lab_name)

                #Check lab info table (ex drop down on Lab Strings):
                #Dropdown inside the lab to see students statuses:
                drop_down_labs[index].click()
                time.sleep(4)


                #Get number of students:
                n_students = len(labs_table[0].find_elements(By.CSS_SELECTOR, "td[class^='MuiTableCell-root MuiTableCell-body column-first_name']"))
                print(n_students, " Students grabbed.")
                for i in range(n_students):

                    #Find name, status and delivery date of each student:
                    #Empty list:
                    student_lab = []

                    student_name = labs_table[0].find_elements(By.CSS_SELECTOR, "td[class^='MuiTableCell-root MuiTableCell-body column-first_name']")[i].text
                    lab_status = labs_table[0].find_elements(By.CSS_SELECTOR, "td[class^='MuiTableCell-root MuiTableCell-body column-status']")[i].text
                    delivery_date = labs_table[0].find_elements(By.CSS_SELECTOR, "td[class^='MuiTableCell-root MuiTableCell-body column-submit_date']")[i].text


                #Append elements into a full row of student info:
                    student_lab.extend([week, each_lab_name, student_name, lab_status, delivery_date])
                    print(student_lab)

                    labs_overall = ( labs_overall.append(pd.Series(
                        student_lab,
                        index = labs_overall.columns),
                        ignore_index=True)
                        )
                print("finished grabbing students info for lab {0} of {1}".format(each_lab_name, week))

                time.sleep(3)
                drop_down_labs[index].click()
                time.sleep(3)
            
            drop_down_weeks.click()
            time.sleep(2)
        
        except:
            print(
                """Got an error.
                Last variables found were :""",student_lab)


        






Week 1
Lab Strings
Required
20  Students grabbed.
['Week 1', 'Lab Strings\nRequired', 'Adrian Flores', 'Delivered', '14/09/2022']


C:\Users\crocs\AppData\Local\Temp\ipykernel_20640\3235245803.py:57: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  labs_overall = ( labs_overall.append(pd.Series(


['Week 1', 'Lab Strings\nRequired', 'Aferdita Zherka Zherka', 'Delivered', '17/09/2022']
['Week 1', 'Lab Strings\nRequired', 'Alicia Andrés', 'Delivered', '25/09/2022']
['Week 1', 'Lab Strings\nRequired', 'Amelie Haberland', 'Delivered', '01/10/2022']
['Week 1', 'Lab Strings\nRequired', 'Angela Arredondo Mendoza', 'Delivered', '14/09/2022']
['Week 1', 'Lab Strings\nRequired', 'Damiano Serra', 'Delivered', '18/09/2022']
['Week 1', 'Lab Strings\nRequired', 'Daniel J Mendez Borges', 'Delivered', '17/09/2022']
['Week 1', 'Lab Strings\nRequired', 'Fabio Blank Da Costa', 'Delivered', '17/09/2022']
['Week 1', 'Lab Strings\nRequired', 'Federico Cavanagh', 'Delivered', '14/09/2022']
['Week 1', 'Lab Strings\nRequired', 'Francisco Javier De Las Heras Jimenez', 'Delivered', '13/09/2022']
['Week 1', 'Lab Strings\nRequired', 'Gregor Seegers', 'Delivered', '14/09/2022']
['Week 1', 'Lab Strings\nRequired', 'Joaquin Valentin', 'Delivered', '23/09/2022']
['Week 1', 'Lab Strings\nRequired', 'Manouk Meilo

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=107.0.5304.88)
Stacktrace:
Backtrace:
	Ordinal0 [0x00A7ACD3+2075859]
	Ordinal0 [0x00A0EE61+1633889]
	Ordinal0 [0x0090B7BD+571325]
	Ordinal0 [0x008F2E1A+470554]
	Ordinal0 [0x0095AA0B+895499]
	Ordinal0 [0x0096AC96+961686]
	Ordinal0 [0x00957136+880950]
	Ordinal0 [0x0092FEFD+720637]
	Ordinal0 [0x00930F3F+724799]
	GetHandleVerifier [0x00D2EED2+2769538]
	GetHandleVerifier [0x00D20D95+2711877]
	GetHandleVerifier [0x00B0A03A+521194]
	GetHandleVerifier [0x00B08DA0+516432]
	Ordinal0 [0x00A1682C+1665068]
	Ordinal0 [0x00A1B128+1683752]
	Ordinal0 [0x00A1B215+1683989]
	Ordinal0 [0x00A26484+1729668]
	BaseThreadInitThunk [0x7665FEF9+25]
	RtlGetAppContainerNamedObjectPath [0x77967BBE+286]
	RtlGetAppContainerNamedObjectPath [0x77967B8E+238]


#### Send student info:

# Create a column with week number as numeric type
labs_overall["week_num"] = labs_overall["week"].apply(lambda x: int(x.split()[1]))

In [ ]:
# Clean the lab_name column to remove "\nRequired"
labs_overall["lab_name"] = labs_overall["lab_name"].apply(lambda x: x.split("\n")[0])

In [ ]:
# Get a list of students and their emails and create a dictionary with both lists
student_names = list(labs_overall["student_name"].unique())
student_names.remove("Manouk Meilof")

student_emails = ["adrianfg88@gmail.com",
"zherka.aferdita@outlook.com",
"aliciaandressalguero@gmail.com",
"amelie.haberland@ironhack.com",
"angela.arredondo99@gmail.com",
"damsvpn@protonmail.com",
"djmborges92@gmail.com",
"fabioblankc@gmail.com",
"fedecavanagh@live.com",
"javierherasjimenez@hotmail.com",
"gregor.seegers@gmail.com",
"quino117@hotmail.com",
"gavalda.marc@gmail.com",
"maria.vazquez.casares@gmail.com",
"maximilian.grempel@gmail.com",
"tanshilei@gmail.com",
"roque.ourense@gmail.com",
"suparna.sahoo@gmail.com",
"anvergara95@gmail.com"]

students_dict = dict(zip(student_names, student_emails))

In [ ]:
for student in student_names:
    
    # Filtering by student
    student_sub = labs_overall[labs_overall["student_name"] == student]
    
    # Filtering by week number
    student_sub_week = student_sub[student_sub["week_num"] <= week_number_email]
    
    # Count delivered and not delivered labs
    submissions = student_sub_week["lab_status"].value_counts()
    delivered = submissions["Delivered"]
    try:
        not_delivered = submissions[""]
    except:
        not_delivered = 0
    
    # Get the lab names of non delivered ones
    pending_labs = student_sub_week[student_sub_week["lab_status"] == ""]["lab_name"].values.tolist()
    pending_labs_strings = "\n".join(pending_labs)
    
    # Calculate the percentage of delivered labs
    lab_percent = round((delivered / (delivered+not_delivered))*100)
                        
    if lab_percent < 80:
        body = f'''Dear {student.split()[0]}, we have been checking your lab sumissions status. From a total of {delivered + not_delivered} labs, you have submitted {delivered}, this represents the {lab_percent}% of the total until now.
Remember that ironhack rules mark 80% as the threshold to complete the course. Don't hesitate in contacting us if you need help to meet that threshold. We will be happy to support you!

The labs that you are missing are:
{pending_labs_strings}
        
Ironhack Teaching Team
        '''
                        
    elif lab_percent == 100:
        body = f'''Dear {student.split()[0]}, we have been checking your lab sumissions status. From a total of {delivered + not_delivered} labs, you have submitted {delivered}, this represents the {lab_percent}% of the total until now.
You have submitted all the labs so far, so congratulations for this hard job!
        
Ironhack Teaching Team
        '''
    
    else:
        body = f'''Dear {student.split()[0]}, we have been checking your lab sumissions status. From a total of {delivered + not_delivered} labs, you have submitted {delivered}, this represents the {lab_percent}% of the total until now.
You are over the 80% threshold marked by Ironhack to complete the course so congratulations! You are properly following the course progress.
Anyway,there are some labs that are still missing:
{pending_labs_strings}

Keep going with the hard job!
        
Ironhack Teaching Team
        '''
        
    email_receiver = "miguelsimonmoya@gmail.com"
    subject = "Ironhack lab status"
    
    em = EmailMessage()
    em["From"] = email_sender
    em["To"] = email_receiver
    em["subject"] = subject
    em.set_content(body)

    context = ssl.create_default_context()

    with smtplib.SMTP_SSL("smtp.gmail.com", context = context) as smtp:
        smtp.login(email_sender, email_password)
        smtp.sendmail(email_sender, email_receiver, em.as_string())
    
                        
    time.sleep(2)
                        